In [3]:
#David Tancredi
#dt959@york.ac.uk - r3dcl1ff@protonmail.com
#Summative assessment code for Advanced Programming module @University of York
import pandas as pd
import numpy as np
import pymongo
import json
from tkinter import *
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from tkinter import filedialog as fd
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)


class UI:
    
    counter = 0 #counter set to 0 to establish state of previous and next buttons
    
    def __init__(self, root):
        #setting up the root window
        self.root=root
        self.root.geometry("1200x800")
        self.client = pymongo.MongoClient("mongodb://localhost:27017/") #establish connection with mongodb
        
        
    
    #=================================Creating Frames for the Application====================================#
       

    #Main window provides options to import csv files or import from MongoDB     
    def main_window(self):
        
        self.menubar = Menu(self.root)
        
        
        # File Menu and Submenu
        self.file = Menu(self.menubar, tearoff = 0)
        self.menubar.add_cascade(label = "Files", menu = self.file)
        self.file.add_command(label = "CSV", command = self.csv_ui)
        self.file.add_command(label = "Mongo", command = self.manipulation_frame)
        self.file.add_command(label = "Exit", command = self.root.destroy)
        #Adding separate commands for different airport categories and joined frequencies 
        self.tasks = Menu(self.menubar, tearoff = 0)
        self.menubar.add_cascade(label = "Actions", menu = self.tasks, state = DISABLED)
        self.tasks.add_command(label = "All UK Airports", state = DISABLED)
        self.tasks.add_command(label = "Large Airports UK", command = self.largeUK, state = DISABLED)
        self.tasks.add_command(label = "Small Airports UK", command = self.smallUK, state = DISABLED)
        self.tasks.add_command(label = "Medium Airports UK", command = self.mediumUK, state = DISABLED)
        self.tasks.add_command(label = "Join Frequencies", command = self.joinFrequencies, state = DISABLED)
        
        self.stats = Menu(self.menubar, tearoff = 0)
        self.menubar.add_cascade(label = "STATS", menu = self.stats, state = DISABLED)
        self.stats.add_command(label = "Summary stats large airports", command = self.summary_Large, state = DISABLED)
        self.stats.add_command(label = "Summary stats freq>100", command = self.summary_Freq_100, state = DISABLED)
        
        self.visual = Menu(self.menubar, tearoff = 0)
        self.menubar.add_cascade(label = "Visualization", menu = self.visual, state = DISABLED)
        self.visual.add_command(label = "Frequencies-small airports", command = self.plotSmallFreq, state = DISABLED)
        self.visual.add_command(label = "Frequencies-all airports", command = self.plotCorrelation, state = DISABLED)
        
        mongoDBlist = self.client.list_database_names()
        # Maintain Database button active only if the database has already been created 
        if "data" in mongoDBlist:
            self.file.entryconfig("CSV", state = NORMAL)
            self.file.entryconfig("Mongo", state = NORMAL)
            
        else:
            self.file.entryconfig("Mongo", state = DISABLED)
            
        #Present the menu
        self.root.config(menu = self.menubar)
            
        
        
        
        
            
            
    # Framing to control the import of csv files
    def csv_ui(self):
        
        #Top frame presents files available for analysis

        self.file_frame=tk.LabelFrame(self.root, text="Files available")
        self.file_frame.place(height=70, width =1150, rely=0, relx=0.01 )

        self.file_frame_label=tk.Label(self.file_frame, text=" airport-frequencies.csv   | airports.csv   | runways.csv")
        self.file_frame_label.place(relx=0.01, rely=0.01)

        self.loadbtn = tk.Button(self.file_frame, text="  Load  ", state = NORMAL, command = self.load_csv_data)
        self.loadbtn.place(relx=0.65, rely=0.01)

        # Frame at the bottom with 3 distinct tabs to display the CSV files

        self.display_frame = tk.LabelFrame(self.root)
        self.display_frame.place(height= 650, width=1150 ,rely=.10, relx= 0.01)

        self.display_frame1 = tk.LabelFrame(self.root)
        self.display_frame2 = tk.LabelFrame(self.root)
        self.display_frame3 = tk.LabelFrame(self.root)
        
        # Button to change view from previous to next and data cleaning option

        self.nextbtn = tk.Button(self.root, text = " Next> ", state=DISABLED, command = self.move_next_btn)
        self.prevbtn = tk.Button(self.root,text = " <Prev ", state=DISABLED, command = self.move_prev_btn)
        self.cleanbtn = tk.Button(self.root, text = ' Clean Data ', command = self.clean_data) 
        
        self.prevbtn.place(relx=0.40, rely=0.94)
        self.nextbtn.place(relx=0.50, rely =0.94)
        
       # ==============Include Treeview for available CSV files=========================#  
        tree_view_airport = self.create_treeview(self.display_frame1)
        tree_view_runway = self.create_treeview(self.display_frame2)
        tree_view_freq = self.create_treeview(self.display_frame3)
        
        self.tree_views=[tree_view_airport,tree_view_runway,tree_view_freq]
        
        
    #Function to convert available CSV files into JSON format
    def csvtoJson(self):     
        
        #Construct a top window to input Database name, collection names for MongoDB
        self.cleanbtn.configure(state = DISABLED)
        self.top = tk.Toplevel(self.root)
        self.top.geometry("700x400")
        self.top.title("Status")
        
          
        #Converting CSV files into JSON format
        self.airport_json = self.df[0].to_json(orient = 'records', indent = 4)
        self.runway_json = self.df[1].to_json(orient = 'records', indent = 4)
        self.freq_json = self.df[2].to_json(orient = 'records', indent = 4)
        
        self.label =tk.Label(self.top, text = "                   CSV files coverted to JSON ")
        self.label.place( relx = 0.2, rely = 0.4)
        
        self.savebtn = tk.Button(self.top, text = " SAVE to MongoDB ", command = self.save_csv_to_mongoDB)  
        self.savebtn.place( relx = 0.4, rely = 0.7)
        
        self.cleanbtn.place_forget()
        
        self.top.grab_set()
        
        
    #Window that designates the names under which converted csv files will be saved into the database
    def save_csv_to_mongoDB(self):
        
        
        self.database = self.client["data"]
        
        self.savebtn.place_forget()
        self.label.place_forget()
            
        #Generating collections
            
        self.collection_airport = self.database["airports"]
        self.collection_runway = self.database["runways"]
        self.collection_freq = self.database["frequencies"]
            
           
        #placing data into individual collections
            
        self.collection_airport.insert_many(json.loads(self.airport_json))
        self.collection_runway.insert_many(json.loads(self.runway_json))
        self.collection_freq.insert_many(json.loads(self.freq_json))
        
        #Redirectiong to the CSV to JSON function
        
        self.label =tk.Label(self.top, text = " Files saved in MongoDB database data as following collections: \n\n airports.csv ==> airport \n runway.csv ==> runways \n airport-frequencies.csv ==> frequencies ")
        self.label.place( relx = 0.1, rely = 0.1)
        
                
        okbtn = tk.Button(self.top, text = " OK ", command = self.manipulation_frame)  
        okbtn.place( relx = 0.5, rely = 0.8)
        
        self.top.grab_set() # Deactivate the main window in the background
        
        
    def manipulation_frame(self):
        
          
               
        # Removing the top window
        self.top.destroy()
        
                
        self.database = self.client.data # Loading data from MongoDB
        self.colllist = self.database.list_collection_names() #Store names of collections in Database
        
        
        #============================= Design ===================================================
        
        # Tab that displays the collections available in the database
        self.collection_list_frame = tk.LabelFrame(self.root)
        self.collection_list_frame.place( height = 50, width = 1199, rely = 0, relx = 0)
                        
        #Adding a blank frame for initial loading of the application 
        self.task2_display_frame_blank = tk.LabelFrame(self.root)
        self.task2_display_frame_blank.place( height = 700, width = 1199, relx = 0.0, rely = 0.07 )
        
        #Frame that displays the results of merged Datasets
        self.task2_display_frame_allUK = tk.LabelFrame(self.root, text = " All UK airports without merged frequencies")  #for "all UK airports"
        self.task2_display_frame_small = tk.LabelFrame(self.root)  #for "small UK airports"
        self.task2_display_frame_medium = tk.LabelFrame(self.root) #for "medium UK airports" 
        self.task2_display_frame_large = tk.LabelFrame(self.root)  #for "large UK airorts"
        
              
        #Frame that displays the results of statistical analysis
        self.task2_display_frame_stats = tk.LabelFrame(self.root)
        
        
        #Frame that displays the results of visualization tasks
        self.task2_display_frame_visual = tk.LabelFrame(self.root)
                     
        #Bottom frame for navigation and exit function
        self.btnframe = tk.LabelFrame( self.root)
        self.btnframe.place(height = 50, width = 1199, relx =0, rely =0.93)
        
        #Text widget that displays the results of statistical analysis 
        self.display_text = tk.Text(self.task2_display_frame_stats)
        
        #Generate a tree view object for the four prepared Datasets
        self.tv_allUK = self.create_treeview(self.task2_display_frame_allUK)
        self.tv_small = self.create_treeview(self.task2_display_frame_small)
        self.tv_medium = self.create_treeview(self.task2_display_frame_medium)
        self.tv_large = self.create_treeview(self.task2_display_frame_large)
        
                
        
        
        
        
        # ===========Preparing the datasets for further manipulation==========================
        # Checking whether the database already contains data in which case collection items will be more than three.
        # Setting an indicator value to help determine data manipulation tasks to be activated
        if len(self.colllist) > 3:
            self.indicator = 1
            
            #Loading datasets from individual collections into dataframes from MongoDB
            pointer = self.database['all_uk_airports'].find()
            self.df_airport = pd.DataFrame(list(pointer))
            
            pointer = self.database['merged_small_airports'].find()
            self.df_airport_small = pd.DataFrame(list(pointer))
            
            pointer = self.database['merged_medium_airports'].find()
            self.df_airport_medium = pd.DataFrame(list(pointer))
            
            pointer = self.database['merged_large_airports'].find()
            self.df_airport_large = pd.DataFrame(list(pointer)) 
            
            
            # Buttons to view three merged dataframes.Save Button is replaced by Exit button
            self.display_task2_bottomframe()
            self.save_btn.configure(text = 'EXIT')
            self.save_btn.configure( command = self.root.destroy)
            
            
        else:
            self.indicator = 0
            self.counter += 1
            
            #Setting variables pointing to airport and frequency collections in MongoDB
            self.collection_airport = self.database['airports']
            
            self.collection_freq = self.database['frequencies']
            
             
               
        #Adding prepared data file names into collection_list_frame if indicator equals one,
        #else revert to initial files
        if self.indicator == 0:
            self.label =tk.Label( self.collection_list_frame, text = " Data Collections Available : airports \t\t |\t\t frequencies")
        else:
            self.label =tk.Label( self.collection_list_frame, text = " Data Collections Available : all_uk_airports | merged_small_airports | merged_medium_airports | merged_large_airports")
                              
        self.label.place(relx = 0.01)
        
                     
        if self.indicator == 0:            
            if self.counter == 1:
                # Activating sub-menu for all UK airports
                self.menubar.entryconfig("Tasks", state = NORMAL)
                self.tasks.entryconfig("All UK airports", state = NORMAL, command = self.allUK)
                            
            elif self.counter == 2:
                #Activating submenu for large UK Airports and disable the rest
                self.tasks.entryconfig("All UK airports", state = DISABLED)
                self.tasks.entryconfig("Large airports UK", state = NORMAL)
            
            elif self.counter == 3 :
                #Activating submenu for small UK airports
                self.tasks.entryconfig("Small airports UK", state = NORMAL)
                self.tasks.entryconfig("Large airports UK", state = DISABLED)
            
            elif self.counter == 4:
                #Activating submenu for medium UK airports
                self.tasks.entryconfig("Medium airports UK", state = NORMAL)
                self.tasks.entryconfig("Small airports UK", state = DISABLED)
            
            elif self.counter == 5:
                #Activating sub menu to join frequencies
                self.tasks.entryconfig("Join Frequencies", state = NORMAL)
                self.tasks.entryconfig("Medium airports UK", state = DISABLED)
            
            elif self.counter == 6:
                #Activating main and sub menus for statistical analysis and and data visualization
                self.menubar.entryconfig("STATS", state = NORMAL)
                self.stats.entryconfig("Summary stats large airports", state = NORMAL)
                self.stats.entryconfig("Summary stats freq>100", state = NORMAL)
                self.menubar.entryconfig("Visualization", state = NORMAL)
                self.visual.entryconfig("Frequencies-small airports", state = NORMAL)
                self.visual.entryconfig("Frequencies-all airports", state = NORMAL)
                self.tasks.entryconfig("Join Frequencies", state = DISABLED)
                self.menubar.entryconfig("Tasks", state = DISABLED)
                
        else:
            
            #Else statement to activate main and sub menu for statistical analysis and data visualization
            self.menubar.entryconfig("STATS", state = NORMAL)
            self.stats.entryconfig("Summary stats large airports", state = NORMAL)
            self.stats.entryconfig("Summary stats freq>100", state = NORMAL)
            self.menubar.entryconfig("Visualization", state = NORMAL)
            self.visual.entryconfig("Frequencies-small airports", state = NORMAL)
            self.visual.entryconfig("Frequencies-all airports", state = NORMAL)
            self.tasks.entryconfig("Join Frequencies", state = DISABLED)
            
        
    
    
    
    # ============Different functions activated once a given command is triggered====================#
    
    
    #Loading CSV files in the application
    def load_csv_data(self):      
    
        self.loadbtn['state'] = DISABLED
        self.nextbtn['state'] = NORMAL  
                   
        #Reading the available CSV files
        self.df=[]
        df_names=['airports.csv', 'runways.csv', 'airport-frequencies.csv']
        df_rows=[]
        
        for i in range(0,len(df_names)):
            self.df.append(pd.read_csv(df_names[i]))
    
            self.tree_views[i]['columns'] = list(self.df[i].columns)
            self.tree_views[i]['show'] = 'headings'

            for column in self.tree_views[i]['columns']:
                self.tree_views[i].heading(column, text=column)
               
            df_rows.append(self.df[i].to_numpy().tolist())
        
            for row in df_rows[i]:
                self.tree_views[i].insert("","end", values=row)
        
        #Placing the "clean" button
        self.cleanbtn.place( relx = 0.80, rely = 0.94)
        
        #Displaying the "airport.csv" file
        
        if self.counter == 0:
            self.display_frame.place_forget()
            self.display_frame1.place(height= 650, width=1150 ,rely=.10, relx= 0.01)
            
        
        
    # Generating treeviews to display dataframes     
    def create_treeview(self, dp_frame):
        tree_view = ttk.Treeview(dp_frame)
        tree_view.place(relheight=1, relwidth=1)
        
        #Adding a scroller to the tree view
        treescrolly = tk.Scrollbar(dp_frame, orient='vertical', command=tree_view.yview) # updates y axis view of the widget
        treescrollx = tk.Scrollbar(dp_frame, orient= 'horizontal', command= tree_view.xview) #updates the x axis view of the widget
        treescrollx.pack(side='bottom', fill='x')
        treescrolly.pack(side='right', fill='y')
        
        return tree_view
    
    #Function for the "Next" buttons
    def move_next_btn(self):
        #Adding a global counter
        self.counter+=1
        if self.counter == 1:
            self.display_frame1.place_forget()
            self.display_frame2.place(height= 650, width=1150 ,rely=.10, relx= 0.01)
            self.prevbtn['state'] = NORMAL
        elif self.counter == 2:
            self.display_frame2.place_forget()
            self.display_frame3.place(height= 650, width=1150 ,rely=.10, relx= 0.01)
            self.nextbtn['state'] = DISABLED
     
    
    #Function for the "Previous" buttons
    def move_prev_btn(self):
        #Adding a global counter
        self.counter-=1
        if self.counter ==1:
            self.display_frame3.place_forget()
            self.display_frame2.place(height= 650, width=1150 ,rely=.10, relx= 0.01)
            self.nextbtn['state'] = NORMAL
        elif self.counter == 0:
            self.display_frame2.place_forget()
            self.display_frame1.place(height= 650, width=1150 ,rely=.10, relx= 0.01)
            self.prevbtn['state'] = DISABLED
    
    
    #Cleaning imported files and inserting them into their respective treeview variables
    def clean_data(self):
        #Deleting columns with missing values
        self.cleaned_df()
        self.counter = 0 #Resetting the counter to zero
        
        #Setting up the layout
        self.display_frame.configure(text = "  Cleaned CSV Files ")
        self.loadbtn.place_forget()
        self.cleanbtn.configure(text = " convert to JSON", command = self.csvtoJson) 
        
        #Cleaning the previous treeview in order to insert the newly cleaned data frames
        for i in range(0,3):
            self.cleaned_treeview(self.tree_views[i])
                    
        
        df_rows=[] #Accomodating records from cleaned data frames
        
        #Inserting rows from the cleaned daframes into the treeview
        for i in range(0,3):            
    
            self.tree_views[i]['columns'] = list(self.df[i].columns)
            self.tree_views[i]['show'] = 'headings'

            for column in self.tree_views[i]['columns']:
                self.tree_views[i].heading(column, text=column)        
    
            df_rows.append(self.df[i].to_numpy().tolist())    
    
            for row in df_rows[i]:
                self.tree_views[i].insert("","end", values=row)
        
        
        if self.counter == 0:
            self.display_frame1.place(height= 650, width=1150 ,rely=.10, relx= 0.01)
            
            
    
     
    #Function to save the prepared data into MongoDB
    def save_prepared_data(self):
        
        #Create respective collections
            
        self.coll_all_uk_airports = self.database['all_uk_airports']
        self.coll_df_airport_large = self.database['merged_large_airports']
        self.coll_df_airport_medium = self.database['merged_medium_airports']
        self.coll_df_airport_small = self.database['merged_small_airports']
            
        #Changing Save button to Exit
        self.save_btn.configure(text = ' EXIT ')
        self.save_btn.configure(command = self.root.destroy)
        
        
        #Converting pandaframes to JSON format
        all_uk_json = self.df_airport.to_json(default_handler = str, orient = 'records', indent = 4)
        large_json = self.df_airport_large.to_json(default_handler = str, orient = 'records', indent = 4)
        medium_json = self.df_airport_medium.to_json(default_handler = str, orient = 'records', indent = 4)
        small_json = self.df_airport_small.to_json(default_handler = str, orient = 'records', indent = 4)
            
        #Inserting data into individual collections
            
        self.coll_all_uk_airports.insert_many(json.loads(all_uk_json))
        self.coll_df_airport_large.insert_many(json.loads(large_json))
        self.coll_df_airport_medium.insert_many(json.loads(medium_json))
        self.coll_df_airport_small.insert_many(json.loads(small_json))
            
        #Message for user once saving data action is complete
        messagebox.showinfo(" Saving into MongoDB ", "Complete", parent = self.root)
        
    #Function to plot communication frequencies of small UK airports
    def plotSmallFreq(self):
        self.task2_display_frame_visual.place( height = 700, width = 1199, relx = 0.0, rely = 0.07 )
        self.task2_display_frame_visual.configure(text = "")
        self.fig = Figure(figsize = (20,20), dpi = 100)  #Defining geometry of figure that will hold the plot
        self.plot1 = self.fig.add_subplot(111)    # Adding a subplot
        
        #Creating a canvas using Tkinter
        
        self.canvas = FigureCanvasTkAgg(self.fig, master = self.task2_display_frame_visual)
        self.plot1.hist(self.df_airport_small['frequency_mhz']) #Plotting the graph
        
        
        self.canvas.draw()
        self.canvas.get_tk_widget().place(relheight = 1, relwidth = 1)
        
        
        
        
     
            
            
    # =============Support functions to assist with various commands and application windows ===========================
    
    
    #The function below parses and cleans columns with null values and at the same time
    #retains columns 'type' and 'iso_country' which will be needed for data manipulation tasks
    #Also deletes rows with null values in 'type' and 'iso_country' attributes
    def cleaned_df(self):
    
        for arg in self.df:
            #Driving out columns with missing values
            delete_columns=arg.columns[arg.isnull().any()].tolist()
    
            #Removing preexisting columns 'type' and ' iso_country' from the list 
            if 'type' in delete_columns:
                delete_columns.remove('type')
        
            if 'iso_country' in delete_columns:
                delete_columns.remove('iso_country')
            
            #Dropping columns with null values
            arg.drop(delete_columns,axis=1,inplace=True)
        
            #Dropping rows where column iso_country is null
            arg.dropna(inplace=True)
            
            
            
    #Function to clean up all widgets employed in the application windows         
    def cleaned_root(self):
        for ele in self.root.winfo_children():
            ele.destroy()
            
    #Function to clean up the treeview       
    def cleaned_treeview(self, treeView_obj):
        treeView_obj.delete(*treeView_obj.get_children())
        
    def display_treeview(self, df):
        
        #Cleaning any old entries in the tree view
        self.cleaned_treeview(self.treeView1)
        self.display_text.place_forget()
        
        #Adding data to the tree view
        self.treeView1["column"] = list(df.columns)
        self.treeView1["show"] = "headings"
        for column in self.treeView1["columns"]:
            self.treeView1.heading(column, text=column) 

        df_rows = df.to_numpy().tolist() #Turning the dataframe into a list of lists with the .tolist() function
        for row in df_rows:
            self.treeView1.insert("", "end", values=row) #Inserting each list into the treeview.
    
    def display_treeview(self, df, treeview):
        #Adding data to tree view
        treeview["column"] = list(df.columns)
        treeview["show"] = "headings"
        for column in treeview["columns"]:
            treeview.heading(column, text=column)

        df_rows = df.to_numpy().tolist() #Turning the dataframe into a list of lists
        for row in df_rows:
            treeview.insert("", "end", values=row) #Inserting each list into the treeview.    

 #=================Functions related to Data Manipulation ========================================# 
    #Data manipulation function for all UK airports 
    def allUK(self):
        self.manipulation_frame()
        
        #Reading airport data from MongodDB                
        pointer = self.collection_airport.find()
        self.df_airport = pd.DataFrame(list(pointer))
        
        
        #Extracting data from all UK airport records
        self.df_airport = self.df_airport[self.df_airport['iso_country'].str.contains('GB')]
        #Dropping redundant colums
        self.df_airport.drop(['latitude_deg','longitude_deg','iso_country','iso_region','scheduled_service'], axis=1, inplace=True)
        
        self.task2_display_frame_blank.place_forget()
        self.task2_display_frame_small.place_forget()
        self.task2_display_frame_medium.place_forget()
        self.task2_display_frame_large.place_forget()
        self.task2_display_frame_stats.place_forget()
        self.task2_display_frame_visual.place_forget()
        
        self.task2_display_frame_allUK.place( height = 700, width = 1199, relx = 0.0, rely = 0.07 )
        self.task2_display_frame_allUK.configure(text = " All UK airports without merged frequencies")
        
        #Displaying the output using treeview
        self.display_treeview(self.df_airport , self.tv_allUK)
       
        
    #Data manipulation function for Large UK airports  
    def largeUK(self):
        self.manipulation_frame()
                
        #Extracting rows with airport type "large" from self.df_airport pandaframe
        self.df_airport_large = self.df_airport[self.df_airport['type'] == 'large_airport']
        
        self.cleaned_treeview(self.tv_large)
        self.task2_display_frame_blank.place_forget()
        self.task2_display_frame_allUK.place_forget()
        self.task2_display_frame_medium.place_forget()
        self.task2_display_frame_small.place_forget()
        self.task2_display_frame_stats.place_forget()
        self.task2_display_frame_visual.place_forget()
        
        self.task2_display_frame_large.place( height = 700, width = 1199, relx = 0.0, rely = 0.07  )
        self.task2_display_frame_large.configure(text = "All large UK airports")
        
        #Displaying the output using tree view
        self.display_treeview(self.df_airport_large , self.tv_large)
        
              
    #Data manipulation function for Small UK airports
    def smallUK(self):
        self.manipulation_frame()
        
        #Extracting rows with airport type "small" from self.df_airport pandaframe
        self.df_airport_small = self.df_airport[self.df_airport['type'] == 'small_airport']
        
        self.cleaned_treeview(self.tv_small)
        self.task2_display_frame_blank.place_forget()
        self.task2_display_frame_allUK.place_forget()
        self.task2_display_frame_medium.place_forget()
        self.task2_display_frame_large.place_forget()
        self.task2_display_frame_stats.place_forget()
        self.task2_display_frame_visual.place_forget()
        
        self.task2_display_frame_small.place( height = 700, width = 1199, relx = 0.0, rely = 0.07  )
        self.task2_display_frame_small.configure(text = "All small UK airports")
        
        #Displaying the output using tree view
        self.display_treeview(self.df_airport_small , self.tv_small)
        
               
    #Data manipulation function for Medium UK airports
    def mediumUK(self): 
        self.manipulation_frame()
        
        #Extracting rows with airport type "medium" from self.df_airport pandaframe
        self.df_airport_medium = self.df_airport[self.df_airport['type'] == 'medium_airport']
        
        self.cleaned_treeview(self.tv_medium)
        self.task2_display_frame_blank.place_forget()
        self.task2_display_frame_allUK.place_forget()
        self.task2_display_frame_small.place_forget()
        self.task2_display_frame_large.place_forget()
        self.task2_display_frame_stats.place_forget()
        self.task2_display_frame_visual.place_forget()
        
        self.task2_display_frame_medium.place( height = 700, width = 1199, relx = 0.0, rely = 0.07 )
        self.task2_display_frame_medium.configure(text = "All medium sized UK airports")
        
        #Displaying the output using treeview
        self.display_treeview(self.df_airport_medium , self.tv_medium)
        
       
    #Data manipulation function for joined airport frequencies
    def joinFrequencies(self):
        self.manipulation_frame()
                
        #Retrieving frequencies from MongoDB
        pointer = self.collection_freq.find()
        self.df_freq = pd.DataFrame(list(pointer))
                
        #Dropping redundant columns
        self.df_freq.drop(['id'], axis=1, inplace=True)
        
        #Merging frequencies with small, medium and large UK records
        self.df_airport_large=pd.merge(self.df_airport_large,self.df_freq, left_on='id', right_on='airport_ref', how='inner')\
                                        .drop(['type_x','airport_ref','airport_ident','type_y'], axis=1)
        self.df_airport_small=pd.merge(self.df_airport_small,self.df_freq, left_on='id', right_on='airport_ref', how='inner')\
                                        .drop(['type_x','airport_ref','airport_ident','type_y'], axis=1)
        self.df_airport_medium=pd.merge(self.df_airport_medium,self.df_freq, left_on='id', right_on='airport_ref', how='inner')\
                                        .drop(['type_x','airport_ref','airport_ident','type_y'], axis=1)
        
        
        self.task2_display_frame_medium.place_forget()
        self.task2_display_frame_allUK.place_forget()
        self.task2_display_frame_small.place_forget()
        self.task2_display_frame_large.place_forget()
        self.task2_display_frame_stats.place_forget()
        self.task2_display_frame_visual.place_forget()
        
        self.task2_display_frame_blank.place( height = 700, width = 1199, relx = 0.0, rely = 0.07  )        
        
        
        #Displaying bottom frame to view the merged datasets and saving option 
        self.display_task2_bottomframe()
        
        
    def display_task2_bottomframe(self):
        # Displays buttons to view the three merged dataframes
        self.merge_label = tk.Label(self.btnframe, text = "View operating frequencies:")
        self.merge_label.place(relx = 0.01)
        self.allUK_btn = tk.Button(self.btnframe, text = "All UK", state = NORMAL, command = self.display_treeview_allUK)
        self.mergeSmall_btn = tk.Button(self.btnframe, text = " Small ", state = NORMAL, command = self.display_mergedSmall_treeview)
        self.mergeMedium_btn = tk.Button(self.btnframe, text = " Medium ", state = NORMAL, command = self.display_mergedMedium_treeview)
        self.mergeLarge_btn = tk.Button(self.btnframe, text = " Large ", state = NORMAL, command = self.display_mergedLarge_treeview)
        self.save_btn = tk.Button(self.btnframe, text = 'SAVE', command = self.save_prepared_data) 
        
        self.allUK_btn.place(relx = 0.3)
        self.mergeSmall_btn.place(relx = 0.4)
        self.mergeMedium_btn.place(relx = 0.5)
        self.mergeLarge_btn.place(relx = 0.6)
        self.save_btn.place (relx = 0.9)
        
   
 #===========Bottom frame functions used to display the merged datasets ========================================       
        
    def increment_counter(self):
       
        self.counter = 1
        self.manipulation_frame()
    
    def display_treeview_allUK(self): #This function displays all UK airports without merged frequencies
        self.cleaned_treeview(self.tv_allUK)
        self.task2_display_frame_blank.place_forget()
        self.task2_display_frame_small.place_forget()
        self.task2_display_frame_medium.place_forget()
        self.task2_display_frame_large.place_forget()
        self.task2_display_frame_stats.place_forget()
        self.task2_display_frame_visual.place_forget()
        
        self.task2_display_frame_allUK.place( height = 700, width = 1199, relx = 0.0, rely = 0.07  )
        self.task2_display_frame_allUK.configure(text = " All UK airports without merged frequencies")
        self.display_treeview(self.df_airport, self.tv_allUK)
                
        
    
    def display_mergedSmall_treeview(self): #This function displays all small UK airports with merged frequencies
        self.cleaned_treeview(self.tv_small)
        self.task2_display_frame_blank.place_forget()
        self.task2_display_frame_allUK.place_forget()
        self.task2_display_frame_medium.place_forget()
        self.task2_display_frame_large.place_forget()
        self.task2_display_frame_stats.place_forget()
        self.task2_display_frame_visual.place_forget()
        
        self.task2_display_frame_small.place( height = 700, width = 1199, relx = 0.0, rely = 0.07 )
        self.task2_display_frame_small.configure(text = "Operating frequencies of small UK airports")
        self.display_treeview(self.df_airport_small, self.tv_small)
       
        
    def display_mergedMedium_treeview(self): #This function displays all medium UK airports with merged frequencies
        self.cleaned_treeview(self.tv_medium)
        self.task2_display_frame_blank.place_forget()
        self.task2_display_frame_allUK.place_forget()
        self.task2_display_frame_small.place_forget()
        self.task2_display_frame_large.place_forget()
        self.task2_display_frame_stats.place_forget()
        self.task2_display_frame_visual.place_forget()
        
        self.task2_display_frame_medium.place( height = 700, width = 1199, relx = 0.0, rely = 0.07  )
        self.task2_display_frame_medium.configure(text = "Operating frequencies of medium UK airports")
        self.display_treeview(self.df_airport_medium, self.tv_medium)
        
        
    def display_mergedLarge_treeview(self): #This function displays all large UK airports with merged frequencies
        self.cleaned_treeview(self.tv_large)
        self.task2_display_frame_blank.place_forget()
        self.task2_display_frame_allUK.place_forget()
        self.task2_display_frame_medium.place_forget()
        self.task2_display_frame_small.place_forget()
        self.task2_display_frame_stats.place_forget()
        self.task2_display_frame_visual.place_forget()
        
        self.task2_display_frame_large.place( height = 700, width = 1199, relx = 0.0, rely = 0.07 )
        self.task2_display_frame_large.configure(text = "Operating frequencies of large UK airports")
        self.display_treeview(self.df_airport_large, self.tv_large)
        

  
     #=====================Functions associated with statistical calculations ==================================#

    def summary_Large(self):  #This function calculates and displays the mean/median/mode of the input dataframe
        
        
        self.task2_display_frame_blank.place_forget()
        self.task2_display_frame_allUK.place_forget()
        self.task2_display_frame_small.place_forget()
        self.task2_display_frame_large.place_forget()
        self.task2_display_frame_medium.place_forget()
        self.task2_display_frame_visual.place_forget()
        
        self.task2_display_frame_stats.configure(text = "Summary stats of frequency for large UK airports")
        self.task2_display_frame_stats.place( height = 700, width = 1199, relx = 0.0, rely = 0.07 )
        
        #Generates and places the Tkinter text widget on task2_display_frame
        self.display_text = tk.Text(self.task2_display_frame_stats)
        self.display_text.place(relwidth = 1, relheight = 1)
        
        #Calculates the mean, median and mode on all large UK airports sets
        mean_df = np.round(self.df_airport_large['frequency_mhz'].mean(),2)
        median_df = np.round(self.df_airport_large['frequency_mhz'].median(),2)
        mode_df = self.df_airport_large['frequency_mhz'].mode()
                
        self.display_text.insert(END, " \n\n\n SUMMARY STATS : frequency for all large UK Airports \n\n MEAN of frequency_mhz   : {} \n MODE of frequency_mhz   : {} \t {} \n MEDIAN of frequency_mhz : {} ".format(mean_df, mode_df[0], mode_df[1], median_df))
        self.display_text.configure(state = DISABLED) #Read-only setting for the text widget
        
    def summary_Freq_100(self):  # Calculates and displays the mean ,mode and median of the input dataframe
        
        self.task2_display_frame_blank.place_forget()
        self.task2_display_frame_allUK.place_forget()
        self.task2_display_frame_small.place_forget()
        self.task2_display_frame_large.place_forget()
        self.task2_display_frame_medium.place_forget()
        self.task2_display_frame_visual.place_forget()
        
        self.task2_display_frame_stats.configure(text = "Summary stats of frequency > 100Mhz for all UK airports")
        self.task2_display_frame_stats.place( height = 700, width = 1199, relx = 0.0, rely = 0.07 )
        
        #Generates and places the Tkinter text widget on task2_display_frame
        self.display_text = tk.Text(self.task2_display_frame_stats)
        self.display_text.place(relwidth = 1, relheight = 1)
        
        #Merges all small, medium and large pandaframes to obtain all frequencies > 100
        temp_df = pd.concat([self.df_airport_small, self.df_airport_medium, self.df_airport_large])
        temp_df = temp_df[temp_df['frequency_mhz']>100]
                        
        #Calculates the mean, median and mode on all large UK airports
        mean_df = np.round(temp_df['frequency_mhz'].mean(),2)
        median_df = np.round(temp_df['frequency_mhz'].median(),2)
        mode_df = temp_df['frequency_mhz'].mode()
                
        self.display_text.insert(END, " \n\n\n SUMMARY STATS : All UK Airports with frequency >100 Mhz \n\n MEAN of frequency_mhz   : {} \n MODE of frequency_mhz   : {}  \n MEDIAN of frequency_mhz : {} ".format(mean_df, mode_df[0], median_df))
        self.display_text.configure(state = DISABLED) #Read-only setting for the text widget
        
    
        
    #This function is used to determine correlation
    def plotCorrelation(self):
        self.task2_display_frame_visual.place( height = 700, width = 1199, relx = 0.0, rely = 0.07 )
        self.task2_display_frame_visual.configure(text = "")
        self.fig = Figure(figsize = (20,20), dpi = 100)  #Defines the figure that will hold the plot
        self.plot1 = self.fig.add_subplot(111)    #Adding a subplot
        
        #Generating a canvas using Tkinter
        
        self.canvas = FigureCanvasTkAgg(self.fig, master = self.task2_display_frame_visual)
        
        #plot
        self.plot1.hist(self.df_airport_small['frequency_mhz'], label = 'small', alpha = 0.5)
        self.plot1.hist(self.df_airport_medium['frequency_mhz'], label = 'medium', alpha = 0.5)
        self.plot1.hist(self.df_airport_large['frequency_mhz'], label = 'large', alpha = 0.5)
        self.plot1.legend()
        
        self.canvas.draw()
        self.canvas.get_tk_widget().place(relheight = 1, relwidth = 1)
        

In [4]:
def main():
    root= tk.Tk()
    
    wiz=UI(root)
    wiz.main_window()
    #wiz.task2_mainwindow()
    
    
    root.mainloop()
    
if __name__ == "__main__":
    main()